# The Right Fit
Overfitting is where a model matches the training data almost perfectly, so when the model is being created, it looks like a near 100% accuracy rate; however, it does poorly in validation and other new data because it is perfectly fitted to the data that it was trained on. Underfitting is the opposite, where a when the model is being built, the accuracies are very high, similar to overfitting, but the model is missing crucial categories that define trends seen in the data and just has a very specific set of comparisons.

Neither are desirable in a model as it greatly reduces the accuracy of the model when it is used on either training data or a larger set of data that was not used for the initial creation of the model. This eliminates the model's usefulness as it will make inaccurate predictions about what it is processing because it is missing a broader understanding of trends/patterns in the data.

For example, a model built using a basic decision tree to determine house prices like the one pictured created below could be overfit if it had so many end nodes that they were each incredibly specific (i.e. 1111 Sherwood Street will sell for \$111,000). When this model is used on data it has never seen before it performs poorly because it was so specified to the data it was trained on. This leads to accuracy suffering as a whole. Conversely, a decision tree with only two end nodes (say \$100,000 and \$200,000) would be inaccurate because it can't recognize broader trends in the data and is focused on only one or two features. Neither makes for a very good ML model. In this chapter you will learn about identifying overfitting/underfitting and how to correct it.

![](https://raw.githubusercontent.com/UncertainQubit/firstrepo/master/R3ywQsR.png)

An example of overfitting can be seen in the model below. When you test the model purely on the data it was trained with it performs extremely well (extremely close estimates to actual housing costs). However, when tested on separate validation data the model has an extremely large margin of error. Below we will explore ways to help identify and correct overfitting and underfitting in your model.

In [10]:
# Code you have previously used to load data
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor


# Path of the file to read
iowa_file_path = 'https://raw.githubusercontent.com/UncertainQubit/firstrepo/master/train.csv'

home_data = pd.read_csv(iowa_file_path)
# Create target object and call it y
y = home_data.SalePrice
# Create X
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[features]

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

# Specify Model
iowa_model = DecisionTreeRegressor(max_leaf_nodes = 10000)
# Fit Model
iowa_model.fit(train_X, train_y)

# Make training predictions and calculate mean absolute error
train_predictions = iowa_model.predict(train_X)
train_mae = mean_absolute_error(train_predictions, train_y)
print("Training Mean Average Error (MAE): ${:,.0f}".format(train_mae))

# Make validation predipoctions and calculate mean absolute error
val_predictions = iowa_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation Mean Average Error (MAE): ${:,.0f}".format(val_mae))

Training Mean Average Error (MAE): $73
Validation Mean Average Error (MAE): $32,541


# Hyperparameter Analysis
One way to identify overfitting/underfitting is if your model performs great with training data but poorly with new testing data which is why it is important to have both a large training dataset and a large testing dataset. One of the primary ways ML engineers work to address this problem is through something called hyperparameter analysis. Essentially this means trying a bunch of different values for various parameters within a model (such as the number of end nodes). This can either be done as a semi-automated process (like we will below) or as a fully automated process using services like AWS Sagemaker. 

# Exercise
We have created the function `get_mae` for you which will generate the MAE for different versions of our model. Code for this function can be viewed in the cell below.

In [11]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

## Step 1: Compare Different Tree Sizes
This loop tries the following values for *max_leaf_nodes* from a set of possible values. By trying a large variety of values and comparing the MAE the loop is able to identify what parameter minimizes inaccuracy. Remember that we are measuring MAE against our *testing* data, this allows us to determine what values minimize underfitting/overfitting and maximize the accuracy of our model when it is actually deployed in a real-world application. We will then print the optimal tree size and store it in the variable *best_tree_size*.

In [12]:
candidate_max_leaf_nodes = list(range(2, 1000))
scores = {leaf_size: get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
best_tree_size = min(scores, key=scores.get)
print('Best max_leaf_nodes = ' + str(best_tree_size))

Best max_leaf_nodes = 82


## Step 2: Fit Model Using All Data
You know the best tree size. If you were going to deploy this model in practice, you would make it even more accurate by using all of the data and keeping that tree size.  That is, you don't need to hold out the validation data now that you've made all your modeling decisions.

In [13]:
import ipywidgets as widgets
widget = widgets.IntSlider(min=1, max=1000, step=10, description='Test', readout=True, continuous_update=True)
display(widget)
final_model = DecisionTreeRegressor(max_leaf_nodes = widget.value, random_state = 0)
final_model.fit(X,y)
cmbd_predictions = final_model.predict(X)
cmbd_mae = mean_absolute_error(cmbd_predictions, y)
print("Combined Mean Average Error (MAE): ${:,.0f}".format(cmbd_mae))

IntSlider(value=1, description='Test', max=1000, min=1, step=10)

ValueError: max_leaf_nodes 1 must be either None or larger than 1

## Recap
While error certainly exists with just a few simple steps we have managed to reduce the error by almost \$14,644 which is a significant margin. In addition, that was all accomplished by changing only one parameter of many. Hyperparameter analysis is an essential tool in any ML engineer's toolkit because it allows for the identification and minimization of underfitting and overfitting which can result in significant penalalties to accuracy when actually deployed in the real world.

### Great Job!